# Data Engineering Project 

## Project Title : Data Modeling with Apache Cassandra

### Gauraw Kishore Tripathy 


In [1]:
# Import Python packages 
import pandas as pd
import re
import os
import glob
import numpy as np
import json
import csv
from cassandra.cluster import Cluster

# Creating list of filepaths to process original event csv data files

#### The below code performs the following operations:

It prints the current working directory using the os.getcwd() function.

It creates a file path variable named filepath by concatenating the current working directory obtained in the previous step with '/event_data'.

It starts a loop that traverses through all the folders and subfolders within the filepath directory using the os.walk() function.

Within each iteration of the loop, it uses the glob.glob() function along with os.path.join() to obtain a list of file paths within the current root directory and its subdirectories. The '*' parameter passed to os.path.join() ensures that all files within the specified directory are included in the list.

Finally, it optionally prints the list of file paths, which is commented out in the provided code.

In summary, this code is printing the current working directory and then collecting a list of file paths for all files within the 'event_data' directory and its subdirectories.

In [2]:
# checking current working directory
print(f"Current working directory : {os.getcwd()}")

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

Current working directory : C:\Users\Gauraw\MCA Sem-3


# Processing the files to create the data file csv that will be used for Apache Casssandra tables

#### The below code performs the following operations:

Initializes an empty list called full_data_rows_list which will be used to store the rows generated from each file.

Iterates over each file path in the file_path_list.

Opens each file using the open() function as a CSV file with the specified encoding.

Creates a CSV reader object using the csv.reader() function to read the file.

Skips the first line of the CSV file using the next() function, assuming it contains headers or irrelevant information.

Iterates over each line in the CSV file using a for loop.

Appends each line (data row) to the full_data_rows_list.

Prints the total number of rows in full_data_rows_list and shows a sample of the data (the first 5 rows).

Registers a custom CSV dialect using csv.register_dialect() specifying that all fields should be quoted and any initial white space should be skipped.

Opens a new CSV file called "event_datafile_new.csv" in write mode with the specified encoding.

Creates a CSV writer object using the csv.writer() function and the custom dialect.

Writes the header row for the desired columns into the new CSV file.

Iterates over each row in full_data_rows_list and writes specific columns to the new CSV file, omitting the row if the first column is empty.

In summary, this code reads data from multiple CSV files, appends the rows to a list, and then writes a subset of the columns from those rows into a new CSV file named "event_datafile_new.csv" using a custom dialect. The specific columns written to the new file are 'artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', and 'userId'.

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

print(f"Total rows : {len(full_data_rows_list)}")
print(f"Sample data:\n {full_data_rows_list[:5]}")

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total rows : 8056
Sample data:
 [['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39'], ['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8'], ['', 'Logged In', 'Kaylee', 'F', '2', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Upgrade', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ['Mr Oizo', 'Logged In', 'Kaylee', 'F', '3', 'Summers', '144.03873', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'Flat 55', '200', '1.54111E+12', '8']]


In [4]:
# checking the number of rows in new event csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Now we are ready to work with the CSV file titled event_datafile_new.csv, located within the Workspace directory. The event_datafile_new.csv contains the following columns:

artist

firstName of user

gender of user

item number in session

last name of user

length of the song

level (paid or free song)

location of the user

sessionId

song title

userId

Showing the denormalized data in the 'event_datafile_new.csv' after the code above is run below:

In [5]:
show_data = pd.read_csv("C:\\Users\\Gauraw\MCA Sem-3\\event_datafile_new.csv")
show_data.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8


# Creating a Cluster

#### Here's what the below code does:

It tries to create a Cluster object by passing a list containing the IP address or hostname of the Cassandra cluster nodes. In this case, it uses the IP address '127.0.0.1', which typically refers to the localhost or the local machine.

It then creates a session object by calling the connect() method on the Cluster object. The connect() method establishes a connection to the Cassandra cluster.

If the connection is successfully established, it prints the message "Connection Established !!".

If there is an exception during the connection attempt, it catches the exception and assigns it to the variable e. It then prints the message "Connection Failed !!" along with the specific error message captured in the e variable.

In summary, this code attempts to connect to a Cassandra cluster running on the local machine using the IP address '127.0.0.1'. If the connection is successful, it prints a success message. If there is an error during the connection attempt, it prints an error message along with the specific error details.






In [6]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1)

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Established !!


# Creating Keyspace

#### Here's what the below code does:

It defines a multiline string variable named keyspace_query. This string contains the CQL (Cassandra Query Language) query for creating a keyspace named "sparkify" with a simple replication strategy and a replication factor of 1.

It tries to execute the keyspace_query using the execute() method on the session object, which represents the connection to the Cassandra cluster.

If the execution is successful, the keyspace is created in Cassandra.

If there is an exception during the execution of the query, it catches the exception and assigns it to the variable e. It then prints the message "Failed to create keyspace!!" along with the specific error details captured in the e variable.

In summary, this code attempts to create a keyspace named "sparkify" in the Cassandra cluster using a specified replication strategy and factor. If the keyspace creation is successful, it proceeds without any output. If there is an error, it prints an error message along with the specific error details.

In [7]:
keyspace_query = """CREATE KEYSPACE IF NOT EXISTS sparkify 
                    with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

# Creating Keyspace
try:
    session.execute(keyspace_query)
except Exception as e:
    print(f"Failed to create keyspace!! Error : {e}")

# Setting Keyspace

In [8]:
# Setting KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

# Now we will create tables to run the following queries. With Apache Cassandra we model the database tables on the queries we want to run.

Below are the queries following which we will build out data model:

1. Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Query 1

##### The below code is responsible for creating a table named "session_item" in Apache Cassandra. Here's what the code does:

It defines a multiline string variable named create_query1. This string contains the CQL (Cassandra Query Language) query for creating the "session_item" table.

The table is defined with the following columns:
artist: text type
song: text type
length: float type
sessionId: int type (used as part of the primary key)
itemInSession: int type (used as part of the primary key)

The primary key for the table consists of the sessionId and itemInSession columns.

It tries to execute the create_query1 using the execute() method on the session object, which represents the connection to the Cassandra cluster.

If the execution is successful, it prints the message "Table Created!!".

If there is an exception during the execution of the query, it catches the exception and assigns it to the variable e. It then prints the message "Table creation failed!!" along with the specific error details captured in the e variable.

In summary, this code attempts to create a table named "session_item" in Cassandra with specific columns and a primary key. If the table creation is successful, it prints a success message. If there is an error, it prints an error message along with the specific error details.

#### For query 1, we need a way to run query on sessionId and itemInSession. So, our primary key must have these columns. We can partition the data on sessionId.

Our Select query : SELECT artist, song, length FROM session_item where sessionId = 338 and itemInSession = 4

Our Primary key will be (sessionId, itemInSession), where sessionId is the partition key and itemInSession is the clustering column.

In [9]:
# Creating table for query1 
create_query1 = """CREATE TABLE IF NOT EXISTS session_item (artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"""

try: 
    session.execute(create_query1)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


#### The below code reads the CSV file named 'event_datafile_new.csv' and inserts its rows into a table named "session_item" in Apache Cassandra. Here's what the code does:

It opens the CSV file named 'event_datafile_new.csv' using the open() function with the specified encoding ('utf8') and assigns it to the variable f.

It creates a CSV reader object named csvreader using the csv.reader() function and passes the opened file (f) as a parameter.

It skips the header row of the CSV file using the next(csvreader) function call, assuming the first line contains column headers and should not be inserted into the Cassandra table.

It starts a loop that iterates over each line in the CSV file using a for loop.

Inside the loop, it prepares an INSERT query for the "session_item" table by constructing a CQL string. The query specifies the columns to insert data into: artist, song, length, sessionId, and itemInSession.

It appends the values from the current line of the CSV file to the query string using string concatenation.

It executes the INSERT query using the execute() method on the session object, passing the query and a tuple of values extracted from the current line of the CSV file.

line[0] represents the artist column value

line[10] represents the song column value

float(line[5]) represents the length column value (converted to a float)

int(line[8]) represents the sessionId column value (converted to an int)

int(line[3]) represents the itemInSession column value (converted to an int)

In summary, this code reads the rows from a CSV file and inserts them into the "session_item" table in Cassandra, mapping the values from specific columns in the CSV file to the corresponding columns in the table.

In [10]:
# Using the event file
file = 'event_datafile_new.csv'

# Reading csv file and inserting rows into cassandra tables.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item (artist, song, length, sessionId, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s) "
        session.execute(query, (line[0], line[10], float(line[5]), int(line[8]), int(line[3])) )

# We will perform a SELECT to verify that the data have been inserted into each table

#### The below code executes a SELECT query on the "session_item" table in Apache Cassandra and retrieves rows that match the specified criteria. Here's what the code does:

It defines a string variable named select_query1 that represents a SELECT query. The query retrieves the artist, song, and length columns from the "session_item" table, filtering the rows based on sessionId = 338 and itemInSession = 4.

It tries to execute the SELECT query using the execute() method on the session object, and assigns the result to the rows variable.

If the execution is successful, the rows variable contains the result of the query.

If there is an exception during the execution of the query, it catches the exception and prints the error message captured in the e variable.

It iterates over the retrieved rows using a for loop.

Inside the loop, it prints each row, which includes the artist, song, and length values.

In summary, this code executes a SELECT query on the "session_item" table to retrieve specific rows that match the criteria sessionId = 338 and itemInSession = 4. It then iterates over the result set and prints the retrieved rows, displaying the artist, song, and length values for each row. This is useful for verifying that the data was correctly inserted into the table and obtaining the specific information for further processing or analysis.

In [11]:
# SELECT statement to verify the data was entered into the table
select_query1 = "SELECT artist, song, length FROM  session_item where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='50', length=495.30731201171875)


# Query 2

#### The below code is responsible for creating a table named "user_session" in Apache Cassandra. Here's what the code does:

It defines a multiline string variable named create_query2. This string contains the CQL (Cassandra Query Language) query for creating the "user_session" table.

The table is defined with the following columns:

sessionId: int type

userId: int type

artist: text type

song: text type

firstName: text type

lastName: text type

itemInSession: int type

The primary key for the table consists of a partition key (sessionId, userId) and a clustering column itemInSession with ascending clustering order (ASC).

It tries to execute the create_query2 using the execute() method on the session object, which represents the connection to the Cassandra cluster.

If the execution is successful, it prints the message "Table Created!!".

If there is an exception during the execution of the query, it catches the exception and assigns it to the variable e. It then prints the message "Table creation failed!!" along with the specific error details captured in the e variable.

In summary, this code attempts to create a table named "user_session" in Cassandra with specific columns and a primary key consisting of a partition key and a clustering column. If the table creation is successful, it prints a success message. If there is an error, it prints an error message along with the specific error details.

#### For query 2, we need a way to run query on sessionId and userId. Also, we need the data sorted on itemInSession. So, our primary key must have these columns. We can partition the data on a composite key (sessionId, userId).

Our Select query : SELECT artist, song, firstName, lastName FROM user_session where sessionId = 182 and userId = 10

Our Primary key will be ((sessionId, userId), itemInSession)), where (sessionId, userId) is the partition key and itemInSession is the clustering column.

Also, we are using the clause - WITH CLUSTERING ORDER BY (itemInSession ASC), to sort our data based on itemInSession

Columns we included in the table : sessionId, userId, artist, song, firstName, lastName, itemInSession

In [12]:
# Creating table for query2 
create_query2 = """CREATE TABLE IF NOT EXISTS user_session (sessionId int, userId int, artist text, song text, firstName text, lastName text, itemInSession int, PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC) """

try: 
    session.execute(create_query2)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


#### The below code reads the CSV file named 'event_datafile_new.csv' and inserts its rows into a table named "user_session" in Apache Cassandra. Here's what the code does:

Assigns the filename 'event_datafile_new.csv' to the variable file.

Opens the CSV file named 'event_datafile_new.csv' using the open() function with the specified encoding ('utf8') and assigns it to the variable f.

Creates a CSV reader object named csvreader using the csv.reader() function and passes the opened file (f) as a parameter.

Skips the header row of the CSV file using the next(csvreader) function call, assuming the first line contains column headers and should not be inserted into the Cassandra table.

Starts a loop that iterates over each line in the CSV file using a for loop.

Inside the loop, it prepares an INSERT query for the "user_session" table by constructing a CQL string. The query specifies the columns to insert data into: sessionId, userId, artist, song, firstName, lastName, and itemInSession.

Appends the values from the current line of the CSV file to the query string using string concatenation.

Executes the INSERT query using the execute() method on the session object, passing the query and a tuple of values extracted from the current line of the CSV file.

int(line[8]) represents the sessionId column value (converted to an int)

int(line[10]) represents the userId column value (converted to an int)

line[0] represents the artist column value

line[9] represents the song column value

line[1] represents the firstName column value

line[4] represents the lastName column value

int(line[3]) represents the itemInSession column value (converted to an int)

In summary, this code reads the rows from a CSV file and inserts them into the "user_session" table in Cassandra, mapping the values from specific columns in the CSV file to the corresponding columns in the table.

In [13]:
file = 'event_datafile_new.csv'

# Reading csv file and inserting rows into cassandra tables.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (sessionId, userId, artist, song, firstName, lastName, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[8]), int(line[10]), line[0], line[9], line[1], line[4], int(line[3])  ) )

#### The below code executes a SELECT query on the "user_session" table in Apache Cassandra and retrieves rows that match the specified criteria. Here's what the code does:

It defines a string variable named select_query2 that represents a SELECT query. The query retrieves the artist, song, firstName, and lastName columns from the "user_session" table, filtering the rows based on sessionId = 182 and userId = 10.

It tries to execute the SELECT query using the execute() method on the session object, and assigns the result to the rows variable.

If the execution is successful, the rows variable contains the result of the query.

If there is an exception during the execution of the query, it catches the exception and prints the error message captured in the e variable.

It iterates over the retrieved rows using a for loop.

Inside the loop, it prints each row, which includes the artist, song, firstName, and lastName values.

In summary, this code executes a SELECT query on the "user_session" table to retrieve specific rows that match the criteria sessionId = 182 and userId = 10. It then iterates over the result set and prints the retrieved rows, displaying the artist, song, firstName, and lastName values for each row. This is useful for obtaining the requested information about the user, session, and music data for further analysis or display.

In [14]:
# SELECT statement to verify the data was entered into the table
select_query2 = "SELECT artist, song, firstName, lastName FROM  user_session where sessionId = 182 and userId = 10"
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


# Query 3

#### The below code is responsible for creating a table named "user_song" in Apache Cassandra. Here's what the code does:

It defines a multiline string variable named create_query3. This string contains the CQL (Cassandra Query Language) query for creating the "user_song" table.

The table is defined with the following columns:

song: text type

userId: int type

firstName: text type

lastName: text type

The primary key for the table consists of a partition key song and a clustering column userId.

It tries to execute the create_query3 using the execute() method on the session object, which represents the connection to the Cassandra cluster.

If the execution is successful, it prints the message "Table Created!!".

If there is an exception during the execution of the query, it catches the exception and assigns it to the variable e. It then prints the message "Table creation failed!!" along with the specific error details captured in the e variable.

In summary, this code attempts to create a table named "user_song" in Cassandra with specific columns and a primary key consisting of a partition key and a clustering column. If the table creation is successful, it prints a success message. If there is an error, it prints an error message along with the specific error details.

#### For query 3, we need a way to run query on song. So, our primary key must have song. Also, the query should be such that it does not contain duplicate users for a song. So we need to model data in such a way that we don't allow duplicate users for a song in our table. This can be acheived by including userId in our primary key.

Our Select query : SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'

Our Primary key will be ((song), userId)), where song is the partition key and userId is the clustering column.

Columns we included in the table : song, userId, firstName, lastName

In [15]:
# Creating table for query3

create_query3 = """CREATE TABLE IF NOT EXISTS user_song (song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))"""

try: 
    session.execute(create_query3)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


#### The below code reads a CSV file named 'event_datafile_new.csv' and inserts its rows into a table named "user_song" in Apache Cassandra. Here's what the code does:

It assigns the filename 'event_datafile_new.csv' to the variable file.

It opens the CSV file named 'event_datafile_new.csv' using the open() function with the specified encoding ('utf8') and assigns it to the variable f.

It creates a CSV reader object named csvreader using the csv.reader() function and passes the opened file (f) as a parameter.

It skips the header row of the CSV file using the next(csvreader) function call, assuming the first line contains column headers and should not be inserted into the Cassandra table.

It starts a loop that iterates over each line in the CSV file using a for loop.

Inside the loop, it prepares an INSERT query for the "user_song" table by constructing a CQL string. The query specifies the columns to insert data into: song, userId, firstName, and lastName.

It appends the values from the current line of the CSV file to the query string using string concatenation.

It executes the INSERT query using the execute() method on the session object, passing the query and a tuple of values extracted from the current line of the CSV file.

line[9] represents the song column value

int(line[10]) represents the userId column value (converted to an int)

line[1] represents the firstName column value

line[4] represents the lastName column value

In summary, this code reads the rows from a CSV file and inserts them into the "user_song" table in Cassandra, mapping the values from specific columns in the CSV file to the corresponding columns in the table.

In [16]:
file = 'event_datafile_new.csv'

# Reading csv file and inserting rows into cassandra tables.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s) "
        session.execute(query, (  line[9], int(line[10]), line[1], line[4] ))

#### The below code executes a SELECT query on the "user_song" table in Apache Cassandra and retrieves rows that match the specified criteria. Here's what the code does:

It defines a string variable named select_query3 that represents a SELECT query. The query retrieves the song, firstName, and lastName columns from the "user_song" table, filtering the rows based on song = 'All Hands Against His Own'.

It tries to execute the SELECT query using the execute() method on the session object, and assigns the result to the rows variable.

If the execution is successful, the rows variable contains the result of the query.

If there is an exception during the execution of the query, it catches the exception and prints the error message captured in the e variable.

It iterates over the retrieved rows using a for loop.

Inside the loop, it prints each row, which includes the song, firstName, and lastName values.

In summary, this code executes a SELECT query on the "user_song" table to retrieve specific rows that match the criteria song = 'All Hands Against His Own'. It then iterates over the result set and prints the retrieved rows, displaying the song, firstName, and lastName values for each row. This is useful for obtaining the names of users who listened to the song 'All Hands Against His Own' from the "user_song" table in Cassandra.

In [17]:
# SELECT statement to verify the data was entered into the table
select_query3 = "SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(select_query3)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


# Drop the tables before closing out the sessions

In [18]:
session.execute("DROP TABLE IF EXISTS sparkify.session_item")
session.execute("DROP TABLE IF EXISTS sparkify.user_session")
session.execute("DROP TABLE IF EXISTS sparkify.user_song")

# Close the session and cluster connection

In [19]:
session.shutdown()
cluster.shutdown()